In [4]:
!python -m pip install torch
!python -m pip install matplotlib
!python -m pip install numpy
!python -m pip install pandas


[notice] A new release of pip available: 22.3 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


     ---------------------------------------- 7.8/7.8 MB 23.8 MB/s eta 0:00:00
     ------------------------------------- 219.8/219.8 kB 13.1 MB/s eta 0:00:00
     ---------------------------------------- 2.2/2.2 MB 23.5 MB/s eta 0:00:00
     ---------------------------------------- 56.0/56.0 kB ? eta 0:00:00
     ---------------------------------------- 2.6/2.6 MB 23.5 MB/s eta 0:00:00
     -------------------------------------- 106.9/106.9 kB 6.0 MB/s eta 0:00:00



[notice] A new release of pip available: 22.3 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip available: 22.3 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip available: 22.3 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [5]:
import torch
import torch.nn as nn
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [9]:
class Node(nn.Module):
    def __init__(self, input_dim = 1, output_dim = 1, hidden_dim = [5,5,5]):
        super(Node, self).__init__()
        layers = []
        
        # Input layer to first hidden layer
        layers.append(nn.Linear(input_dim, hidden_dim[0]))
        layers.append(nn.ReLU())  # Activation function
        
        # Hidden layers
        for i in range(1, len(hidden_dim)):
            layers.append(nn.Linear(hidden_dim[i-1], hidden_dim[i]))
            layers.append(nn.ReLU())  # Activation function
        
        # Output layer
        layers.append(nn.Linear(hidden_dim[-1], output_dim))
        
        # Combining all the layers into a sequential model
        self.model = nn.Sequential(*layers)
    
    def forward(self, x):
        return self.model(x)
        
mlp = Node()
print(mlp)

Node(
  (model): Sequential(
    (0): Linear(in_features=1, out_features=5, bias=True)
    (1): ReLU()
    (2): Linear(in_features=5, out_features=5, bias=True)
    (3): ReLU()
    (4): Linear(in_features=5, out_features=5, bias=True)
    (5): ReLU()
    (6): Linear(in_features=5, out_features=1, bias=True)
  )
)


In [ ]:
class KAN_Layer(nn.Module)
    def __init__(self, input_dim, width, output_dim):
        nodes = []
        for i in range(0, width):
            nodes.append(Node(n))